In [ ]:
folder = "sentences/"
filename = "newhome"
filetoread = '{}{}.txt'.format(folder,filename)
sentences_list = []
with open(filetoread, 'r', encoding='utf-8') as f:
    sentences_list = f.readlines()

In [ ]:
import pandas as pd
import re
import numpy as np
import logging

In [ ]:
logging.basicConfig(level=logging.ERROR)

In [ ]:
df = pd.read_csv('verbs-all-new.csv', engine='python', encoding='utf-8')
df = df.replace(np.nan, '', regex=True)

In [ ]:
df.head()

In [ ]:
df[df["root"]== "tener"].iloc[0]

In [ ]:
df.iloc[6]

In [ ]:
haber_present = df[df['root']=='haber'].iloc[:,1:6].to_numpy().flatten().tolist()

In [ ]:
def removeEmpty(lis):
    return list(filter(None, lis))

In [ ]:
def fetchVerbs(tense, df):
    tense_list = df.filter(regex=("{}.*".format(tense))).to_numpy().flatten().tolist()
    tense_list = removeEmpty(tense_list)
    return tense_list

In [ ]:
estar_present = fetchVerbs("indicative_present", df[df['root']=='estar'])
estar_preterite = fetchVerbs("indicative_preterite", df[df['root']=='estar'])
estar_imperfect = fetchVerbs("indicative_imperfect", df[df['root']=='estar'])

In [ ]:
moods = ['indicative', 'perfect']
tenses = ['present', 'preterite', 'imperfect', 'conditional', 'future', 'past']

In [ ]:
import itertools
names = list(map('_'.join, itertools.product(moods, tenses)))

In [ ]:
for i, item in enumerate(names, start=0):
    print(i,item)

In [ ]:
conjugationMap = {}
for i in names:
    conjugationMap[i] = fetchVerbs(i, df)

conjugationMap.keys()

In [ ]:
df = df[df['root']!='haber']

In [ ]:
f = open('{}.html'.format(filename),'w')

In [ ]:
notdone = [True]*len(sentences_list)

In [ ]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
def doFormatting(sentence, start, end, color):
    sentence = sentence[:start] + "<b style=\"color:{};\">".format(color) + sentence[start:end] + "</b>" + sentence[end:]
    logging.debug("Sentence doFormatting [%s]", sentence)
    return sentence

In [ ]:
def isDone(sentence):
    if "</b>" in sentence:
        return True
    return False

In [ ]:
def findTense(verbList, sentence, color):
    formattedSentence = sentence
    logging.debug("verbList in findTense(): [%s]", verbList)
    for one_verb in verbList:
        logging.debug("verb in findtense(): [%s]", one_verb)
        a = re.search(r'\b({})\b'.format(one_verb), formattedSentence.lower(),  re.UNICODE)
        
        if (a is not None):
            logging.debug("Found match [%s] verb [%s]", a, one_verb)
            formattedSentence = doFormatting(formattedSentence, a.start(), a.end(), color)
            logging.debug("Formatted sentence [%s]", formattedSentence)
    return formattedSentence

In [ ]:
import cssmaker
cssmaker.do()

In [ ]:
tense, color = cssmaker.getTenseColor()

In [ ]:
colormaphex = dict(zip(tense, color))
colormaphex

https://www.w3schools.com/colors/colors_names.asp

In [ ]:
with open('tabl2.html', 'r') as file:
    message = file.read().replace('\n', '')

In [ ]:
# sentences_list = [" Sí, yo he amado a mi esposa desde ese día. "]

In [ ]:
estar_preterite

In [ ]:
for i in range(len(sentences_list)):
    one_sentence = sentences_list[i]
    # check for -ndo ending
    # progressive +ndo
    ending_in_ndo = re.search(r'\b(\w+)\s(\w+ndo)\b', one_sentence.lower(),  re.UNICODE)
    ending_in_iado = re.search(r'\b(\w+)\s(\w+[ia]do)\b', one_sentence.lower(),  re.UNICODE)
    if ending_in_ndo is not None:
        logging.debug("ending_in_ndo: {%s} | {%s}", ending_in_ndo.group(1), ending_in_ndo.group(2))
        #check the previous word
        start = ending_in_ndo.start()
        end = ending_in_ndo.end()
        if ending_in_ndo.group(1) in estar_present:
            one_sentence = doFormatting(one_sentence, start, end, colormaphex["#ipresent"])
        elif ending_in_ndo.group(1) in estar_preterite:
            one_sentence = doFormatting(one_sentence, start, end, colormaphex["#ipreterite"])

        elif ending_in_ndo.group(1) in estar_imperfect:
            one_sentence = doFormatting(one_sentence, start, end, colormaphex["#iimperfect"])
#         else:
#             one_sentence = doFormatting(one_sentence, one_sentence.index(ending_in_ndo.group(2)),\
#                                       one_sentence.index(ending_in_ndo.group(2))+len(ending_in_ndo.group(2)) , \
#                                         dict_["indicative_progressive_imperfect"])
    
    one_sentence = findTense(conjugationMap["indicative_present"], one_sentence, colormaphex["#ipresent"])
    one_sentence = findTense(conjugationMap["indicative_preterite"], one_sentence, colormaphex["#ipreterite"])
    one_sentence = findTense(conjugationMap["indicative_imperfect"], one_sentence, colormaphex["#iimperfect"])
    one_sentence = findTense(conjugationMap["indicative_conditional"], one_sentence, colormaphex["#iconditional"])
    one_sentence = findTense(conjugationMap["indicative_future"], one_sentence, colormaphex["#ifuture"])
    
    one_sentence = findTense(conjugationMap["perfect_present"], one_sentence, colormaphex["#ppresent"])

    logging.debug("ending_in_iado: [%s]", ending_in_iado)
    
    if ending_in_iado is not None:
        one_sentence = remove_tags(one_sentence)
        logging.debug("ending_in_iado {%s} | {%s}", ending_in_iado.group(1), ending_in_iado.group(2))
        start = ending_in_iado.start()
        end = ending_in_iado.end()
        if ending_in_iado.group(1) in estar_present:
            one_sentence = doFormatting(one_sentence,start, end, colormaphex["#ppresent"])
        elif ending_in_iado.group(1) in estar_preterite:
            one_sentence = doFormatting(one_sentence, start, end, colormaphex["#ipreterite"])

        elif ending_in_iado.group(1) in estar_imperfect:
            one_sentence = doFormatting(one_sentence, start, end, colormaphex["#ipreterite"])
        elif ending_in_iado.group(1) in haber_present:
            one_sentence = doFormatting(one_sentence, start, end, colormaphex["#ppresent"])

    
    formated = one_sentence + "<br/>"
    message += formated

In [ ]:
message += "</p></body></html>"

In [ ]:
f.write(message)
f.close()